# 构建 Rust 包

In [3]:
%%bash

pwd

cargo build

/mnt/ssd/geneplus/develop/liushen/DDRTree/Python/tests/sparse_module


   Compiling pyo3-build-config v0.18.3
   Compiling pyo3-ffi v0.18.3
   Compiling pyo3 v0.18.3
   Compiling sparse_module v0.1.0 (/mnt/ssd/geneplus/develop/liushen/DDRTree/Python/tests/sparse_module)
    Finished dev [unoptimized + debuginfo] target(s) in 5.03s


In [3]:
%%bash
maturin develop

🔗 Found pyo3 bindings
🐍 Found CPython 3.12 at /opt/miniconda/envs/r42/bin/python
   Compiling sparse_module v0.1.0 (/mnt/ssd/geneplus/develop/liushen/DDRTree/Python/tests/sparse_module)
    Finished dev [unoptimized + debuginfo] target(s) in 1.14s
📦 Built wheel for CPython 3.12 to /tmp/.tmpZo5qC3/sparse_module-0.1.0-cp312-cp312-linux_x86_64.whl
⚠️ Warning: pip raised a warning running ["-m", "pip", "--disable-pip-version-check", "install", "--no-deps", "--force-reinstall", "/tmp/.tmpZo5qC3/sparse_module-0.1.0-cp312-cp312-linux_x86_64.whl"]:
DEPRECATION: Loading egg at /opt/miniconda/envs/r42/lib/python3.12/site-packages/example-0.1-py3.12-linux-x86_64.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330



✏️  Setting installed package as editable


🛠 Installed sparse_module-0.1.0


In [2]:
import numpy as np
from scipy.sparse import csr_matrix

# Set a random seed for reproducibility
np.random.seed(42)

# Generate sparse matrix A (3 x 3)
row_a = np.random.randint(0, 3, size=5)  # Random row indices
col_a = np.random.randint(0, 3, size=5)  # Random column indices
data_a = np.random.rand(5)  # Random non-zero values
matrix_a = csr_matrix((data_a, (row_a, col_a)), shape=(3, 3))

# Generate sparse matrix B (3 x 2) - Fixed shape
row_b = np.random.randint(0, 3, size=4)  # Random row indices
col_b = np.random.randint(0, 2, size=4)  # Random column indices
data_b = np.random.rand(4)  # Random non-zero values
matrix_b = csr_matrix((data_b, (row_b, col_b)), shape=(3, 2))

# Print the generated sparse matrices
print("Matrix A (CSR format):")
print(matrix_a.toarray())  # Convert to dense format for viewing
print("Matrix B (CSR format):")
print(matrix_b.toarray())

# Extract CSR matrix parameters
data_a = matrix_a.data
indices_a = matrix_a.indices
indptr_a = matrix_a.indptr

data_b = matrix_b.data
indices_b = matrix_b.indices
indptr_b = matrix_b.indptr

# Placeholder for calling the Rust multiply_csr_matrices function
def multiply_csr_matrices(data_a, indices_a, indptr_a, data_b, indices_b, indptr_b):
    # Print the parameters passed to the Rust function
    print("Calling Rust function with the following parameters:")
    print("Matrix A data:", data_a)
    print("Matrix A indices:", indices_a)
    print("Matrix A indptr:", indptr_a)
    print("Matrix B data:", data_b)
    print("Matrix B indices:", indices_b)
    print("Matrix B indptr:", indptr_b)
    
    # Simulated result using SciPy (replace with Rust function call)
    mat_a = csr_matrix((data_a, indices_a, indptr_a), shape=(3, 3))
    mat_b = csr_matrix((data_b, indices_b, indptr_b), shape=(3, 2))
    return mat_a.dot(mat_b)

# Call the matrix multiplication
result = multiply_csr_matrices(data_a, indices_a, indptr_a, data_b, indices_b, indptr_b)

# Print the multiplication result
print("Result of A * B:")
print(result.toarray())  # Convert to dense format for viewing


Matrix A (CSR format):
[[0.         0.         0.88676064]
 [0.         0.         0.        ]
 [0.05808361 0.60111501 0.70807258]]
Matrix B (CSR format):
[[0.         0.        ]
 [0.72317416 0.82899867]
 [0.         0.        ]]
Calling Rust function with the following parameters:
Matrix A data: [0.88676064 0.05808361 0.60111501 0.70807258]
Matrix A indices: [2 0 1 2]
Matrix A indptr: [0 1 1 4]
Matrix B data: [0.72317416 0.82899867]
Matrix B indices: [0 1]
Matrix B indptr: [0 0 2 2]
Result of A * B:
[[0.         0.        ]
 [0.         0.        ]
 [0.43471084 0.49832355]]


In [3]:
import numpy as np
from scipy.sparse import csr_matrix, random
from sparse_module import multiply_csr_matrices

def generate_random_sparse_matrix(rows, cols, density=0.01, format='csr'):
    """Generates a random sparse matrix with specified dimensions and density."""
    return random(rows, cols, density=density, format=format, dtype=np.float64)

def test_csr_matrix_multiply_with_random_matrices():
    np.random.seed(42)  # Set a seed for reproducibility

    # Define dimensions for matrices A and B
    rows_a = np.random.randint(5, 20)  # Random number of rows for matrix A
    cols_a_rows_b = np.random.randint(5, 20)  # Shared dimension for A (columns) and B (rows)
    cols_b = np.random.randint(5, 20)  # Random number of columns for matrix B

    # Generate random sparse matrices A and B
    matrix_a = generate_random_sparse_matrix(rows_a, cols_a_rows_b)
    matrix_b = generate_random_sparse_matrix(cols_a_rows_b, cols_b)

    # Print matrix shapes
    print("Matrix A shape:", matrix_a.shape)
    print("Matrix B shape:", matrix_b.shape)

    # Call Rust function for CSR matrix multiplication
    rust_result = multiply_csr_matrices(matrix_a, matrix_b)

    # Compute the expected result using SciPy's dot method
    python_result = matrix_a.dot(matrix_b)

    # Convert Rust's result back to a CSR matrix for comparison
    rust_csr_result = csr_matrix(
        (rust_result['data'], rust_result['indices'], rust_result['indptr']),
        shape=rust_result['shape']
    )

    # Compare the results
    if np.allclose(rust_csr_result.toarray(), python_result.toarray(), atol=1e-8):
        print("Test passed: The multiplication results are equal.")
    else:
        print("Test failed: The multiplication results differ.")
        # Provide additional details
        diff = rust_csr_result - python_result
        print("Difference matrix (non-zero elements):", diff.nnz)
        print(diff)

if __name__ == "__main__":
    # Run the test case
    test_csr_matrix_multiply_with_random_matrices()


Matrix A shape: (11, 8)
Matrix B shape: (8, 17)
Matrix A: Data length: 1, Indices length: 1, Indptr length: 12
Shape A: (11, 8)


thread '<unnamed>' panicked at /root/.cargo/registry/src/index.crates.io-6f17d22bba15001f/sprs-0.8.1/src/sparse/csmat.rs:519:59:
called `Result::unwrap()` on an `Err` value: IllegalArguments("Indptr length does not match dimension")
note: run with `RUST_BACKTRACE=1` environment variable to display a backtrace


PanicException: called `Result::unwrap()` on an `Err` value: IllegalArguments("Indptr length does not match dimension")

In [5]:
import sparse_module  # 导入生成的 Rust 扩展模块
import scipy.sparse as sp
import numpy as np


# 矩阵 A (3x4)
data_a = np.array([1.0, 2.0, 3.0])  # 非零元素
indices_a = np.array([1, 2, 3])  # 非零元素的列索引
indptr_a = np.array([0, 1, 2, 3, 3])  # 行的指针，长度应该是 行数 + 1
shape_a = (3, 4)

# 矩阵 B (4x3)
data_b = np.array([4.0, 5.0, 6.0])  # 非零元素
indices_b = np.array([1, 2, 0])  # 非零元素的列索引
indptr_b = np.array([0, 1, 2, 3, 3])  # 行的指针，长度应该是 行数 + 1
shape_b = (4, 3)

# 创建稀疏矩阵
csr_matrix_a = sp.csr_matrix((data_a, indices_a, indptr_a), shape=shape_a)
csr_matrix_b = sp.csr_matrix((data_b, indices_b, indptr_b), shape=shape_b)

# 调用 Rust 中的稀疏矩阵乘法函数
result = sparse_module.multiply_csr_matrices(csr_matrix_a, csr_matrix_b)

# 输出结果
print("Multiplication Result:")
print(f"Data: {result['data']}")
print(f"Indices: {result['indices']}")
print(f"Indptr: {result['indptr']}")
print(f"Shape: {result['shape']}")

ValueError: index pointer size 5 should be 4